# Relatório dos dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurações iniciais
pd.set_option('display.max_columns', None)

# Caminhos das bases
BASE_TREINO_PATH = "/kaggle/input/data-elasticidade/base_treino.xlsx"
BASE_TESTE_PATH = "/kaggle/input/data-elasticidade/base_teste.xlsx"

# Função para carregar bases
def carregar_base(path, nome):
    if os.path.exists(path):
        return pd.read_excel(path)
    else:
        print(f"Arquivo {nome} não encontrado em {path}")
        return None

# Carregar bases
base_treino = carregar_base(BASE_TREINO_PATH, "base_treino.xlsx")
base_teste = carregar_base(BASE_TESTE_PATH, "base_teste.xlsx")

if base_treino is None or base_teste is None:
    raise ValueError("Uma das bases essenciais não foi carregada!")

# Função para gerar relatório exploratório
def gerar_relatorio(base, nome_base):
    print(f"\n=== Relatório Exploratório da {nome_base} ===")

    # 1. Informações básicas
    print("\n1. Informações Básicas:")
    print(f"Número de linhas: {base.shape[0]}")
    print(f"Número de colunas: {base.shape[1]}")

    # 2. Valores ausentes
    print("\n2. Percentual de Valores Ausentes por Coluna:")
    missing = base.isnull().mean() * 100
    print(missing[missing > 0].sort_values(ascending=False))

    # 3. Estatísticas descritivas
    print("\n3. Estatísticas Descritivas das Variáveis Numéricas:")
    num_cols = ['share', 'vendas_in_hl', 'preco_medio_in_hl', 'price_index_tt',
                'distr_numerica_dn', 'distr_pond_vendedora', 'volume_total', 'trend']
    desc = base[num_cols].describe()
    print(desc)

    # 4. Consistência do Share por UF e Período
    print("\n4. Soma do Share por UF e Período (deve ser ~100%):")
    if 'mes_ano' in base.columns:
        share_sum = base.groupby(['uf', 'mes_ano'])['share'].sum()
    else:
        share_sum = base.groupby('uf')['share'].sum()
    print(share_sum.describe())

    # 5. Observações por UF
    print("\n5. Número de Observações por UF:")
    uf_counts = base['uf'].value_counts()
    print(uf_counts)

    # 6. Distribuição Temporal
    print("\n6. Distribuição Temporal:")
    if 'dt_completa' in base.columns:
        base['dt_completa'] = pd.to_datetime(base['dt_completa'], format='%d/%m/%Y', errors='coerce')
        print(f"Período: {base['dt_completa'].min()} a {base['dt_completa'].max()}")
    elif 'mes_ano' in base.columns:
        print(f"Período: {base['mes_ano'].min()} a {base['mes_ano'].max()}")
    elif 'trend' in base.columns:
        print(f"Trend: {base['trend'].min()} a {base['trend'].max()}")

    # 7. Outliers (usando IQR)
    print("\n7. Detecção de Outliers (IQR):")
    for col in num_cols:
        Q1 = base[col].quantile(0.25)
        Q3 = base[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = base[(base[col] < Q1 - 1.5 * IQR) | (base[col] > Q3 + 1.5 * IQR)][col]
        if not outliers.empty:
            print(f"{col}: {len(outliers)} outliers detectados")

    # 8. Visualizações
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=base[num_cols], orient='h')
    plt.title(f"Distribuição das Variáveis Numéricas - {nome_base}")
    plt.savefig(f"boxplot_{nome_base.lower().replace(' ', '_')}.png")
    plt.close()

    plt.figure(figsize=(12, 6))
    base['uf'].value_counts().plot(kind='bar', title=f"Distribuição por UF - {nome_base}")
    plt.savefig(f"uf_dist_{nome_base.lower().replace(' ', '_')}.png")
    plt.close()

# Gerar relatórios
gerar_relatorio(base_treino, "Base de Treino")
gerar_relatorio(base_teste, "Base de Teste")

# Comparação entre bases
print("\n=== Comparação entre Bases ===")
print("Colunas exclusivas da Base de Treino:", set(base_treino.columns) - set(base_teste.columns))
print("Colunas exclusivas da Base de Teste:", set(base_teste.columns) - set(base_treino.columns))

# Exportar relatório como CSV
# relatorio_treino = base_treino.describe(include='all')
# relatorio_teste = base_teste.describe(include='all')
# relatorio_treino.to_csv("relatorio_base_treino.csv")
# relatorio_teste.to_csv("relatorio_base_teste.csv")
# print("\nRelatórios exportados como 'relatorio_base_treino.csv' e 'relatorio_base_teste.csv'")


=== Relatório Exploratório da Base de Treino ===

1. Informações Básicas:
Número de linhas: 114124
Número de colunas: 45

2. Percentual de Valores Ausentes por Coluna:
fator_hl    0.124426
dtype: float64

3. Estatísticas Descritivas das Variáveis Numéricas:
               share   vendas_in_hl  preco_medio_in_hl  price_index_tt  \
count  114124.000000  114124.000000      114124.000000   114124.000000   
mean        0.007728     250.732272        1112.229651        1.121239   
std         0.014576     575.793170         393.749723        0.399042   
min         0.000071       1.160000          72.072727        0.064940   
25%         0.000806      22.840000         795.054677        0.798410   
50%         0.002066      61.010000        1032.401827        1.031985   
75%         0.007114     208.092500        1369.521317        1.381222   
max         0.208426   12477.150000        3736.842105        3.851600   

       distr_numerica_dn  distr_pond_vendedora   volume_total          tre

# Dados

In [2]:
df_exploracao_teste = pd.read_excel('/kaggle/input/data-elasticidade/base_teste.xlsx')
df_exploracao_teste

,tag_prod,tag_mkt,material_embalagem,faixa_tamanho,embalagem,mes_ano,ano,mes,semana,dt_completa,geo,uf,canal,sub_canal,fabricante,marca,sub_marca,segmento,sub_segmento,long_prod,tipo_de_malte,tipo_cerveja,vendas_in_hl,vendas_valor,preco_medio_in_hl,distr_numerica_dn,distr_pond_vendedora,share,vendas_tt,price_index_tt,price_index_ts,price_index_tss,preco_medio_tt,preco_medio_ts,preco_medio_tss,volume_total,vendas_normalizada_in_hl,vendas_valor_normalizada_in_hl,trend,concorrencia,preco_normalizado_in_hl,fator_hl,pack,embalagem_pack,fabricantexsegmento
0,P000000000002014174000000002000156011804,M000000000010002331200000000000020140123,LT.,350 A 400 ML,DESC.,2024-6,2024,6,22,2024-06-02,GEO SP,SP,OFF,MODERNO,PETROPOLIS,PETRA,ORIGEM,MAINSTREAM,MAINSTREAM,PETRA ORIGEM PURO MALTE LAGER LT. 350 A 400 ML...,PURO MALTE,LAGER,2072.32,1661826,801.915727,49,73,0.031374,66052.95,0.73547,0.88149,0.91588,1090.349000,909.729558,875.564050,68506.0592,2072.32,1661826,42,CONCORRENCIA,801.915727,0.00350,LATA 350,OW,PETROPOLIS-CORE TOTAL
1,P000000000002014174000000002000156012090,M000000000010002331200000000000020140123,LT.,350 A 400 ML,DESC.,2024-6,2024,6,22,2024-06-02,GEO SP,SP,OFF,MODERNO,AMBEV,BRAHMA,ZERO ALCOOL,MAINSTREAM,MAINSTREAM,BRAHMA ZERO ALCOOL COMUM LAGER LT. 350 A 400 M...,COMUM,LAGER,235.02,264281,1124.504298,64,89,0.003558,66052.95,1.03133,1.23609,1.28432,1090.349000,909.729558,875.564050,68506.0592,235.02,264281,42,AMBEV,1124.504298,0.00350,LATA 350,OW,AMBEV-CORE TOTAL
2,P000000000002014174000000002000156012093,M000000000010002331200000000000020140123,V.,300 A 310 ML,RET.,2024-6,2024,6,22,2024-06-02,GEO SP,SP,OFF,MODERNO,AMBEV,BRAHMA,CHOPP,MAINSTREAM,MAINSTREAM,BRAHMA CHOPP COMUM PILSEN V. 300 A 310 ML RET.,COMUM,PILSEN,253.96,196215,772.621673,15,36,0.003845,66052.95,0.70860,0.84929,0.88243,1090.349000,909.729558,875.564050,68506.0592,253.96,196215,42,AMBEV,772.621673,0.00300,RGB 300,RGB,AMBEV-CORE TOTAL
3,P000000000002014174000000002000156012097,M000000000010002331200000000000020140123,LT.,251 A 299 ML,DESC.,2024-6,2024,6,22,2024-06-02,GEO SP,SP,OFF,MODERNO,AMBEV,BRAHMA,CHOPP,MAINSTREAM,MAINSTREAM,BRAHMA CHOPP COMUM PILSEN LT. 251 A 299 ML DESC.,COMUM,PILSEN,207.89,199956,961.835586,47,62,0.003147,66052.95,0.88214,1.05728,1.09853,1090.349000,909.729558,875.564050,68506.0592,207.89,199956,42,AMBEV,961.835586,0.00269,LATA 269,OW,AMBEV-CORE TOTAL
4,P000000000002014174000000002000156012129,M000000000010002331200000000000020140123,LT.,300 A 310 ML,DESC.,2024-6,2024,6,22,2024-06-02,GEO SP,SP,OFF,MODERNO,AMBEV,SKOL,SKOL,MAINSTREAM,MAINSTREAM,SKOL COMUM PILSEN LT. 300 A 310 ML DESC.,COMUM,PILSEN,288.92,225889,781.839263,8,17,0.004374,66052.95,0.71705,0.85942,0.89295,1090.349000,909.729558,875.564050,68506.0592,288.92,225889,42,AMBEV,781.839263,0.00310,LATA 310,OW,AMBEV-CORE TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3579,P000000000002014174000000002000156011666,M000000000010002335700000000000020140623,LT.,251 A 299 ML,DESC.,2024-6,2024,6,25,2024-06-23,GEO CO,CO,OFF,MODERNO,HEINEKEN,DEVASSA,BEM LOURA,MAINSTREAM,MAINSTREAM,DEVASSA BEM LOURA COMUM PILSEN LT. 251 A 299 M...,COMUM,PILSEN,7.44,7159,962.231183,2,3,0.000738,10077.89,0.83591,1.25585,1.25585,1151.117446,766.197168,766.197168,11200.5133,7.44,7159,42,CONCORRENCIA,962.231183,0.00269,LATA 269,OW,HEINEKEN-VALUE
3580,P000000000002014174000000002000156011836,M000000000010002335700000000000020140623,LT.,350 A 400 ML,DESC.,2024-6,2024,6,25,2024-06-23,GEO CO,CO,OFF,MODERNO,PETROPOLIS,ITAIPAVA,ITAIPAVA,MAINSTREAM,MAINSTREAM,ITAIPAVA COMUM PILSEN LT. 350 A 400 ML DESC.,COMUM,PILSEN,21.06,16309,774.406458,24,41,0.002090,10077.89,0.67274,1.01071,1.01071,1151.117446,766.197168,766.197168,11200.5133,21.06,16309,42,CONCORRENCIA,774.406458,0.00350,LATA 350,OW,PETROPOLIS-VALUE
3581,P000000000002014174000000002000156012252,M00000000001000233570000000000002

In [3]:
df_exploracao_treino = pd.read_excel('/kaggle/input/data-elasticidade/base_treino.xlsx')
df_exploracao_treino

,material_embalagem,faixa_tamanho,embalagem,mes_ano,ano,mes,semana,dt_completa,tag_mkt,geo,uf,canal,sub_canal,tag_prod,fabricante,marca,sub_marca,segmento,sub_segmento,long_prod,tipo_de_malte,tipo_cerveja,vendas_in_hl,vendas_valor,preco_medio_in_hl,distr_numerica_dn,distr_pond_vendedora,share,vendas_tt,price_index_tt,price_index_ts,price_index_tss,preco_medio_tt,preco_medio_ts,preco_medio_tss,volume_total,vendas_normalizada_in_hl,vendas_valor_normalizada_in_hl,trend,concorrencia,preco_normalizado_in_hl,fator_hl,pack,embalagem_pack,fabricantexsegmento
0,LT.,251 A 299 ML,DESC.,2022-1,2022,1,52,2022-01-02,M000000000010002331200000000000020140127,GEO NO,NO,OFF,MODERNO,P000000000002014174000000002000156011654,HEINEKEN,DEVASSA,DEVASSA,MAINSTREAM,MAINSTREAM,DEVASSA PURO MALTE LAGER LT. 251 A 299 ML DESC.,PURO MALTE,LAGER,476.11,330064,693.251560,25,43,0.015191,31341.19,0.75847,0.89385,0.97193,914.010444,775.579115,713.275368,41742.3936,476.11,330064,13,CONCORRENCIA,693.251560,0.00269,LATA 269,OW,HEINEKEN-CORE TOTAL
1,LT.,350 A 400 ML,DESC.,2022-1,2022,1,52,2022-01-02,M000000000010002331200000000000020140127,GEO NO,NO,OFF,MODERNO,P000000000002014174000000002000156011681,HEINEKEN,AMSTEL,AMSTEL,MAINSTREAM,MAINSTREAM,AMSTEL PURO MALTE LAGER LT. 350 A 400 ML DESC.,PURO MALTE,LAGER,1124.09,802855,714.226619,60,89,0.035866,31341.19,0.78142,0.92089,1.00133,914.010444,775.579115,713.275368,41742.3936,1124.09,802855,13,CONCORRENCIA,714.226619,0.00350,LATA 350,OW,HEINEKEN-CORE TOTAL
2,LT.,451 A 499 ML,DESC.,2022-1,2022,1,52,2022-01-02,M000000000010002331200000000000020140127,GEO NO,NO,OFF,MODERNO,P000000000002014174000000002000156011682,HEINEKEN,AMSTEL,AMSTEL,MAINSTREAM,MAINSTREAM,AMSTEL PURO MALTE LAGER LT. 451 A 499 ML DESC.,PURO MALTE,LAGER,430.67,276084,641.056958,32,54,0.013741,31341.19,0.70137,0.82655,0.89875,914.010444,775.579115,713.275368,41742.3936,430.67,276084,13,CONCORRENCIA,641.056958,0.00473,LATA 473,OW,HEINEKEN-CORE TOTAL
3,V.,350 A 400 ML,DESC.,2022-1,2022,1,52,2022-01-02,M000000000010002331200000000000020140127,GEO NO,NO,OFF,MODERNO,P000000000002014174000000002000156011800,PETROPOLIS,PETRA,ORIGEM,MAINSTREAM,MAINSTREAM,PETRA ORIGEM PURO MALTE LAGER V. 350 A 400 ML ...,PURO MALTE,LAGER,41.70,41757,1001.366906,36,49,0.001331,31341.19,1.09557,1.29112,1.40390,914.010444,775.579115,713.275368,41742.3936,41.70,41757,13,CONCORRENCIA,1001.366906,0.00350,BIG LN,OW,PETROPOLIS-CORE TOTAL
4,LT.,350 A 400 ML,DESC.,2022-1,2022,1,52,2022-01-02,M000000000010002331200000000000020140127,GEO NO,NO,OFF,MODERNO,P000000000002014174000000002000156011804,PETROPOLIS,PETRA,ORIGEM,MAINSTREAM,MAINSTREAM,PETRA ORIGEM PURO MALTE LAGER LT. 350 A 400 ML...,PURO MALTE,LAGER,871.40,644646,739.781960,58,80,0.027804,31341.19,0.80938,0.95384,1.03716,914.010444,775.579115,713.275368,41742.3936,871.40,644646,13,CONCORRENCIA,739.781960,0.00350,LATA 350,OW,PETROPOLIS-CORE TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114119,LT.,451 A 499 ML,DESC.,2024-5,2024,5,20,2024-05-19,M000000000010002331200000000000020140133,GEO NE,NE,OFF,MODERNO,P000000000002014174000000002000156012254,AMBEV,ANTARCTICA,SUB ZERO,MAINSTREAM,MAINSTREAM,ANTARCTICA SUB ZERO COMUM PILSEN LT. 451 A 499...,COMUM,PILSEN,14.86,1389,93.472409,6,4,0.001477,10060.13,0.08581,0.14233,0.14233,1089.285824,656.730684,656.730684,13251.6882,14.86,1389,41,AMBEV,93.472409,0.00473,LATA 473,OW,AMBEV-VALUE
114120,LT.,451 A 499 ML,DESC.,2024-5,2024,5,20,2024-05-19,M000000000010002331200000000000020140133,GEO NE,NE,OFF,MODERNO,P000000000002014174000000002000156012258,AMBEV,NOSSA PERNAMBUCO,NOSSA PERNAMBUCO,MAINSTREAM,MAINSTREAM,NOSSA PERNAMBUCO COMUM PILSEN LT. 451 A 499 ML...,COMUM,PILSEN,27.46,18359,668.572469,8,22,0.002730,10060.13,0.61377,1.01803,1.01803,1089.285824,656.730684,656.730684,13251.6882,27.46,18359,41,AMBEV,668.572469,0.00473,LATA 473,OW,AMBEV-VALUE
114121,LT.,451

# Teste Modelo

In [4]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# Configurações iniciais
pd.set_option('display.max_columns', None)
np.random.seed(42)

# Função para carregar dados
def load_data(path):
    df = pd.read_excel(path, parse_dates=['dt_completa'])
    col_preco = [c for c in df.columns if 'preco' in c.lower() and 'medio' in c.lower() and 'tt' not in c.lower() and 'ts' not in c.lower() and 'tss' not in c.lower()][0]
    if col_preco != 'preco_medio_in_hl':
        df.rename(columns={col_preco: 'preco_medio_in_hl'}, inplace=True)
    
    if 'marca_original' in df.columns and 'marca' not in df.columns:
        df.rename(columns={'marca_original': 'marca', 'sub_marca_original': 'sub_marca'}, inplace=True)
    
    return df.sort_values('dt_completa')

# Função de winsorização
def winsorize_series(series, limits=[0.1, 0.1]):
    lower, upper = series.quantile(limits[0]), series.quantile(1 - limits[1])
    return np.clip(series, lower, upper), (lower, upper)

# Feature Engineering ajustado
def create_features(df_train, df_test=None):
    df_train = df_train.copy()
    if df_test is not None:
        df_test = df_test.copy()
    
    required_cols = ['preco_medio_in_hl', 'distr_numerica_dn', 'share', 'vendas_in_hl', 'price_index_tt']
    for col in required_cols:
        if col not in df_train.columns:
            raise KeyError(f"Coluna obrigatória ausente: {col}")
    
    # Winsorização baseada apenas em treino
    for col in ['share', 'preco_medio_in_hl', 'vendas_in_hl']:
        train_wins, limits = winsorize_series(df_train[col])
        df_train[col] = train_wins
        if df_test is not None:
            df_test[col] = np.clip(df_test[col], limits[0], limits[1])
    
    # Features explicativas (sem ln_share)
    df_train['price_squared'] = df_train['preco_medio_in_hl'] ** 2
    df_train['month'] = df_train['dt_completa'].dt.month
    for lag in [1, 2, 3, 4]:
        df_train[f'price_lag{lag}'] = df_train.groupby('uf')['preco_medio_in_hl'].shift(lag)
        df_train[f'volume_lag{lag}'] = df_train.groupby('uf')['vendas_in_hl'].shift(lag)
    
    # Target encoding
    if 'marca' in df_train.columns:
        marca_enc = df_train.groupby('marca')['share'].mean().to_dict()
        df_train['marca_target_enc'] = df_train['marca'].map(marca_enc)
    if 'segmento' in df_train.columns:
        segmento_enc = df_train.groupby('segmento')['share'].mean().to_dict()
        df_train['segmento_target_enc'] = df_train['segmento'].map(segmento_enc)
    
    # Aplicar ao teste
    if df_test is not None:
        df_test['price_squared'] = df_test['preco_medio_in_hl'] ** 2
        df_test['month'] = df_test['dt_completa'].dt.month
        for lag in [1, 2, 3, 4]:
            df_test[f'price_lag{lag}'] = df_test.groupby('uf')['preco_medio_in_hl'].shift(lag)
            df_test[f'volume_lag{lag}'] = df_test.groupby('uf')['vendas_in_hl'].shift(lag)
        if 'marca' in df_test.columns:
            marca_mean = np.mean(list(marca_enc.values()))
            df_test['marca_target_enc'] = df_test['marca'].map(marca_enc).fillna(marca_mean)
        if 'segmento' in df_test.columns:
            segmento_mean = np.mean(list(segmento_enc.values()))
            df_test['segmento_target_enc'] = df_test['segmento'].map(segmento_enc).fillna(segmento_mean)
        return df_train, df_test
    
    return df_train

# Modelo otimizado por região
def train_optimized_model(X_train, y_train, uf=None):
    if uf == 'RJ_ES':
        model = CatBoostRegressor(iterations=1000, depth=4, learning_rate=0.05, l2_leaf_reg=5, silent=True)
    elif uf == 'NO':
        model = XGBRegressor(learning_rate=0.1, max_depth=4, n_estimators=500, reg_lambda=15, subsample=0.7, colsample_bytree=0.6, random_state=42)
    else:
        param_grid = {'learning_rate': [0.03, 0.05], 'max_depth': [5, 7], 'n_estimators': [300, 500], 'reg_lambda': [5, 10]}
        model = XGBRegressor(objective='reg:squarederror', random_state=42)
        tscv = TimeSeriesSplit(n_splits=5)
        grid_search = GridSearchCV(model, param_grid, cv=tscv, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
    
    model.fit(X_train, y_train)
    return model

# Pipeline principal
def main():
    base_treino = load_data("/kaggle/input/data-elasticidade/base_treino.xlsx")
    base_teste = load_data("/kaggle/input/data-elasticidade/base_teste.xlsx")
    
    # Diagnóstico básico para SUL
    sul_treino = base_treino[base_treino['uf'] == 'SUL']
    print(f"\nDiagnóstico SUL (treino):")
    print(f"Número de observações: {len(sul_treino)}")
    print(f"Missing values: {sul_treino[['preco_medio_in_hl', 'share', 'vendas_in_hl', 'price_index_tt']].isnull().sum().to_string()}")
    print(f"Estatísticas básicas:\n{sul_treino[['preco_medio_in_hl', 'share', 'vendas_in_hl', 'price_index_tt']].describe()}")
    
    # Features ajustadas (sem ln_share)
    base_features = ['price_squared', 'price_lag1', 'price_lag2', 'month', 'volume_lag1', 
                     'marca_target_enc', 'segmento_target_enc', 'price_index_tt']
    
    resultados = []
    ufs = base_treino['uf'].unique()
    
    for uf in ufs:
        print(f"\nProcessando {uf}...")
        
        train_data = base_treino[base_treino['uf'] == uf].copy()
        test_data = base_teste[base_teste['uf'] == uf].copy()
        
        # Feature engineering
        train_data, test_data = create_features(train_data, test_data)
        
        features = [f for f in base_features if f in train_data.columns and f in test_data.columns]
        X_train = train_data[features].fillna(0)
        y_train = np.log(train_data['share'] + 1e-6)
        X_test = test_data[features].fillna(0)
        y_test = np.log(test_data['share'] + 1e-6)
        
        model = train_optimized_model(X_train, y_train, uf)
        
        y_pred = model.predict(X_test)
        r2 = r2_score(np.exp(y_test), np.exp(y_pred))
        wmape_val = np.sum(np.abs(np.exp(y_pred) - np.exp(y_test))) / np.sum(np.exp(y_test))
        rmse = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
        
        resultados.append({
            'UF': uf,
            'R² Teste': r2,
            'WMAPE Teste': wmape_val,
            'RMSE Teste': rmse,
            'Modelo': 'CatBoost' if uf == 'RJ_ES' else 'XGBoost'
        })
        print(f"{uf}: R²={r2:.4f}, WMAPE={wmape_val:.4f}, RMSE={rmse:.4f}")
    
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv("resultados_finais.csv", index=False)
    print("\nResultados Finais:")
    print(df_resultados.sort_values('R² Teste', ascending=False).to_string(index=False))

if __name__ == "__main__":
    main()


Diagnóstico SUL (treino):
Número de observações: 20175
Missing values: preco_medio_in_hl    0
share                0
vendas_in_hl         0
price_index_tt       0
Estatísticas básicas:
       preco_medio_in_hl         share  vendas_in_hl  price_index_tt
count       20175.000000  20175.000000  20175.000000    20175.000000
mean         1068.495368      0.006245    304.029980        1.106272
std           382.315642      0.011404    584.432361        0.395042
min           241.760344      0.000137      5.490000        0.251890
25%           766.409128      0.000779     37.090000        0.795760
50%           962.425088      0.002035     96.990000        0.984680
75%          1309.949518      0.005873    285.770000        1.360880
max          2997.205503      0.163753   9690.230000        3.194160

Processando NO...
NO: R²=0.3271, WMAPE=0.6164, RMSE=0.0076

Processando SUL...
SUL: R²=0.5318, WMAPE=0.4578, RMSE=0.0036

Processando CO...
CO: R²=0.5765, WMAPE=0.4515, RMSE=0.0048

Processand

In [5]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

# Configurações iniciais
pd.set_option('display.max_columns', None)
np.random.seed(42)

# Função para carregar dados
def load_data(path):
    df = pd.read_excel(path, parse_dates=['dt_completa'])
    col_preco = [c for c in df.columns if 'preco' in c.lower() and 'medio' in c.lower() and 'tt' not in c.lower() and 'ts' not in c.lower() and 'tss' not in c.lower()][0]
    if col_preco != 'preco_medio_in_hl':
        df.rename(columns={col_preco: 'preco_medio_in_hl'}, inplace=True)
    
    if 'marca_original' in df.columns and 'marca' not in df.columns:
        df.rename(columns={'marca_original': 'marca', 'sub_marca_original': 'sub_marca'}, inplace=True)
    
    return df.sort_values('dt_completa')

# Função de winsorização
def winsorize_series(series, limits=[0.05, 0.05]):
    lower, upper = series.quantile(limits[0]), series.quantile(1 - limits[1])
    return np.clip(series, lower, upper), (lower, upper)

# Feature Engineering ajustado
def create_features(df_train, df_test=None):
    df_train = df_train.copy()
    if df_test is not None:
        df_test = df_test.copy()
    
    required_cols = ['preco_medio_in_hl', 'distr_numerica_dn', 'share', 'vendas_in_hl', 'price_index_tt']
    for col in required_cols:
        if col not in df_train.columns:
            raise KeyError(f"Coluna obrigatória ausente: {col}")
    
    # Winsorização
    for col in ['share', 'preco_medio_in_hl', 'vendas_in_hl', 'distr_numerica_dn']:
        train_wins, limits = winsorize_series(df_train[col])
        df_train[col] = train_wins
        if df_test is not None:
            df_test[col] = np.clip(df_test[col], limits[0], limits[1])
    
    # Features explicativas
    df_train['price_squared'] = df_train['preco_medio_in_hl'] ** 2
    df_train = pd.concat([df_train, pd.get_dummies(df_train['dt_completa'].dt.month, prefix='month')], axis=1)
    df_train['price_index_preco'] = df_train['price_index_tt'] * df_train['preco_medio_in_hl']
    df_train['trend'] = df_train.groupby('uf').cumcount()
    df_train['price_dist_interaction'] = df_train['preco_medio_in_hl'] * df_train['distr_numerica_dn']
    
    # Lags expandidos
    for lag in [1, 2, 3, 4]:
        df_train[f'price_lag{lag}'] = df_train.groupby('uf')['preco_medio_in_hl'].shift(lag)
        df_train[f'volume_lag{lag}'] = df_train.groupby('uf')['vendas_in_hl'].shift(lag)
        df_train[f'share_lag{lag}'] = df_train.groupby('uf')['share'].shift(lag)
    
    # Target encoding
    if 'marca' in df_train.columns:
        marca_enc = df_train.groupby('marca')['share'].mean().to_dict()
        df_train['marca_target_enc'] = df_train['marca'].map(marca_enc)
    if 'segmento' in df_train.columns:
        segmento_enc = df_train.groupby('segmento')['share'].mean().to_dict()
        df_train['segmento_target_enc'] = df_train['segmento'].map(segmento_enc)
    
    # Aplicar ao teste
    if df_test is not None:
        df_test['price_squared'] = df_test['preco_medio_in_hl'] ** 2
        df_test = pd.concat([df_test, pd.get_dummies(df_test['dt_completa'].dt.month, prefix='month')], axis=1)
        df_test['price_index_preco'] = df_test['price_index_tt'] * df_test['preco_medio_in_hl']
        df_test['trend'] = df_test.groupby('uf').cumcount()
        df_test['price_dist_interaction'] = df_test['preco_medio_in_hl'] * df_test['distr_numerica_dn']
        for lag in [1, 2, 3, 4]:
            df_test[f'price_lag{lag}'] = df_test.groupby('uf')['preco_medio_in_hl'].shift(lag)
            df_test[f'volume_lag{lag}'] = df_test.groupby('uf')['vendas_in_hl'].shift(lag)
            df_test[f'share_lag{lag}'] = df_test.groupby('uf')['share'].shift(lag)
        if 'marca' in df_test.columns:
            marca_mean = np.mean(list(marca_enc.values()))
            df_test['marca_target_enc'] = df_test['marca'].map(marca_enc).fillna(marca_mean)
        if 'segmento' in df_test.columns:
            segmento_mean = np.mean(list(segmento_enc.values()))
            df_test['segmento_target_enc'] = df_test['segmento'].map(segmento_enc).fillna(segmento_mean)
        return df_train, df_test
    
    return df_train

# Verificar elasticidade
def check_elasticity(model, X_test, y_pred, uf):
    X_test_sim = X_test.copy()
    X_test_sim['preco_medio_in_hl'] *= 1.01  # +1%
    y_pred_up = model.predict(X_test_sim)
    X_test_sim['preco_medio_in_hl'] *= 0.99/1.01  # -1%
    y_pred_down = model.predict(X_test_sim)
    price_up_effect = np.mean(y_pred_up - y_pred)
    price_down_effect = np.mean(y_pred_down - y_pred)
    print(f"{uf} - Impacto com +1% preço: {price_up_effect:.4f} (esperado: negativo)")
    print(f"{uf} - Impacto com -1% preço: {price_down_effect:.4f} (esperado: positivo)")
    return price_up_effect < 0 and price_down_effect > 0

# Modelo otimizado por região
def train_optimized_model(X_train, y_train, uf=None):
    # Definir TimeSeriesSplit fora do bloco condicional
    tscv = TimeSeriesSplit(n_splits=5)
    
    if uf == 'RJ_ES':
        model = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.03, l2_leaf_reg=5, silent=True)
    elif uf == 'NO':
        model = XGBRegressor(learning_rate=0.05, max_depth=5, n_estimators=700, reg_lambda=10, subsample=0.8, colsample_bytree=0.7, random_state=42)
    else:
        param_dist = {
            'learning_rate': [0.01, 0.03, 0.05, 0.1],
            'max_depth': [3, 5, 7, 9],
            'n_estimators': [300, 500, 700, 1000],
            'reg_lambda': [1, 5, 10, 20],
            'subsample': [0.7, 0.8, 0.9],
            'colsample_bytree': [0.6, 0.7, 0.8]
        }
        model = XGBRegressor(objective='reg:squarederror', random_state=42)
        search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=30, cv=tscv, 
                                    scoring='r2', n_jobs=-1, random_state=42)
        search.fit(X_train, y_train)
        model = search.best_estimator_
    
    # Validação cruzada interna
    cv_scores = []
    for train_idx, val_idx in tscv.split(X_train):
        X_cv_train, X_cv_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_cv_train, y_cv_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        model.fit(X_cv_train, y_cv_train)
        y_cv_pred = model.predict(X_cv_val)
        cv_r2 = r2_score(np.exp(y_cv_val), np.exp(y_cv_pred))
        cv_scores.append(cv_r2)
    print(f"{uf} - Média R² Validação Cruzada: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")
    
    model.fit(X_train, y_train)
    return model

# Pipeline principal
def main():
    base_treino = load_data("/kaggle/input/data-elasticidade/base_treino.xlsx")
    base_teste = load_data("/kaggle/input/data-elasticidade/base_teste.xlsx")
    
    # Diagnóstico básico para SUL
    sul_treino = base_treino[base_treino['uf'] == 'SUL']
    print(f"\nDiagnóstico SUL (treino):")
    print(f"Número de observações: {len(sul_treino)}")
    print(f"Missing values: {sul_treino[['preco_medio_in_hl', 'share', 'vendas_in_hl', 'price_index_tt']].isnull().sum().to_string()}")
    print(f"Estatísticas básicas:\n{sul_treino[['preco_medio_in_hl', 'share', 'vendas_in_hl', 'price_index_tt']].describe()}")
    
    # Features expandidas, incluindo preco_medio_in_hl
    base_features = ['preco_medio_in_hl', 'price_squared', 'price_lag1', 'price_lag2', 'price_lag3', 'price_lag4', 
                     'volume_lag1', 'volume_lag2', 'volume_lag3', 'volume_lag4', 
                     'share_lag1', 'share_lag2', 'share_lag3', 'share_lag4', 
                     'marca_target_enc', 'segmento_target_enc', 'price_index_tt', 
                     'price_index_preco', 'trend', 'distr_numerica_dn', 'price_dist_interaction'] + \
                     [f'month_{i}' for i in range(1, 13)]
    
    resultados = []
    ufs = base_treino['uf'].unique()
    
    for uf in ufs:
        print(f"\nProcessando {uf}...")
        
        train_data = base_treino[base_treino['uf'] == uf].copy()
        test_data = base_teste[base_teste['uf'] == uf].copy()
        
        # Feature engineering
        train_data, test_data = create_features(train_data, test_data)
        
        features = [f for f in base_features if f in train_data.columns and f in test_data.columns]
        X_train = train_data[features].fillna(train_data[features].median())
        y_train = np.log(train_data['share'] + 1e-6)
        X_test = test_data[features].fillna(train_data[features].median())
        y_test = np.log(test_data['share'] + 1e-6)
        
        model = train_optimized_model(X_train, y_train, uf)
        
        y_pred = model.predict(X_test)
        r2 = r2_score(np.exp(y_test), np.exp(y_pred))
        wmape_val = np.sum(np.abs(np.exp(y_pred) - np.exp(y_test))) / np.sum(np.exp(y_test))
        rmse = np.sqrt(mean_squared_error(np.exp(y_test), np.exp(y_pred)))
        
        # Verificar relação negativa
        is_negative = check_elasticity(model, X_test, y_pred, uf)
        
        resultados.append({
            'UF': uf,
            'R² Teste': r2,
            'WMAPE Teste': wmape_val,
            'RMSE Teste': rmse,
            'Relação Negativa': is_negative,
            'Modelo': 'CatBoost' if uf == 'RJ_ES' else 'XGBoost'
        })
        print(f"{uf}: R²={r2:.4f}, WMAPE={wmape_val:.4f}, RMSE={rmse:.4f}, Relação Negativa={is_negative}")
        if not is_negative:
            print(f"Aviso: {uf} não respeita a relação negativa entre preço e share!")
    
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv("resultados_finais.csv", index=False)
    print("\nResultados Finais:")
    print(df_resultados.sort_values('R² Teste', ascending=False).to_string(index=False))

if __name__ == "__main__":
    main()


Diagnóstico SUL (treino):
Número de observações: 20175
Missing values: preco_medio_in_hl    0
share                0
vendas_in_hl         0
price_index_tt       0
Estatísticas básicas:
       preco_medio_in_hl         share  vendas_in_hl  price_index_tt
count       20175.000000  20175.000000  20175.000000    20175.000000
mean         1068.495368      0.006245    304.029980        1.106272
std           382.315642      0.011404    584.432361        0.395042
min           241.760344      0.000137      5.490000        0.251890
25%           766.409128      0.000779     37.090000        0.795760
50%           962.425088      0.002035     96.990000        0.984680
75%          1309.949518      0.005873    285.770000        1.360880
max          2997.205503      0.163753   9690.230000        3.194160

Processando NO...
NO - Média R² Validação Cruzada: 0.8088 ± 0.0198
NO - Impacto com +1% preço: -0.0046 (esperado: negativo)
NO - Impacto com -1% preço: 0.0090 (esperado: positivo)
NO: R²=0.684

# Validação modelo

In [6]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import warnings
warnings.filterwarnings('ignore')

# Configurações iniciais
pd.set_option('display.max_columns', None)
np.random.seed(42)

# Funções originais (inalteradas)
def load_data(path):
    df = pd.read_excel(path, parse_dates=['dt_completa'])
    col_preco = [c for c in df.columns if 'preco' in c.lower() and 'medio' in c.lower() and 'tt' not in c.lower() and 'ts' not in c.lower() and 'tss' not in c.lower()][0]
    if col_preco != 'preco_medio_in_hl':
        df.rename(columns={col_preco: 'preco_medio_in_hl'}, inplace=True)
    if 'marca_original' in df.columns and 'marca' not in df.columns:
        df.rename(columns={'marca_original': 'marca', 'sub_marca_original': 'sub_marca'}, inplace=True)
    return df

def winsorize_series(series, limits=[0.1, 0.1]):
    return np.clip(series, series.quantile(limits[0]), series.quantile(1 - limits[1]))

def create_features(df, uf=None):
    df = df.copy()
    required_cols = ['preco_medio_in_hl', 'distr_numerica_dn', 'share', 'vendas_in_hl', 'price_index_tt']
    for col in required_cols:
        if col not in df.columns:
            raise KeyError(f"Coluna obrigatória ausente: {col}")
    df['ln_share'] = np.log(df['share'] + 1e-6)
    df['price_squared'] = df['preco_medio_in_hl'] ** 2
    if 'dt_completa' in df.columns and 'uf' in df.columns:
        df = df.sort_values(['uf', 'dt_completa'])
        df['month'] = df['dt_completa'].dt.month
        for lag in [1, 2, 3, 4]:
            df[f'share_lag{lag}'] = df.groupby('uf')['share'].shift(lag)
            df[f'price_lag{lag}'] = df.groupby('uf')['preco_medio_in_hl'].shift(lag)
            df[f'volume_lag{lag}'] = df.groupby('uf')['vendas_in_hl'].shift(lag)
    if 'marca' in df.columns:
        df['marca_target_enc'] = df.groupby('marca')['share'].transform('mean')
    if 'segmento' in df.columns:
        df['segmento_target_enc'] = df.groupby('segmento')['share'].transform('mean')
    return df

def train_optimized_model(X_train, y_train, uf=None):
    if uf == 'RJ_ES':
        model = CatBoostRegressor(iterations=300, depth=3, learning_rate=0.01, l2_leaf_reg=20, silent=True)
    elif uf == 'NO':
        model = XGBRegressor(learning_rate=0.03, max_depth=3, n_estimators=200, reg_lambda=30, subsample=0.9, colsample_bytree=0.5, random_state=42)
    else:
        param_grid = {'learning_rate': [0.01, 0.03], 'max_depth': [3], 'n_estimators': [100, 200], 'reg_lambda': [20, 30]}
        model = XGBRegressor(objective='reg:squarederror', random_state=42)
        tscv = TimeSeriesSplit(n_splits=5)
        grid_search = GridSearchCV(model, param_grid, cv=tscv, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        model = grid_search.best_estimator_
    model.fit(X_train, y_train)
    return model

# Função de validação
def validate_model():
    base_treino = load_data("/kaggle/input/data-elasticidade/base_treino.xlsx")
    base_teste = load_data("/kaggle/input/data-elasticidade/base_teste.xlsx")
    base_treino = create_features(base_treino)
    base_teste = create_features(base_teste)
    
    for col in ['share', 'preco_medio_in_hl', 'vendas_in_hl']:
        base_treino[col] = winsorize_series(base_treino[col])
        base_teste[col] = winsorize_series(base_teste[col])
    
    # Reduzir features para focar no preço
    base_features = ['price_squared', 'price_lag1', 'price_lag2', 'price_index_tt']
    features = [f for f in base_features if f in base_treino.columns and f in base_teste.columns]
    print(f"Features usadas: {features}")
    
    ufs = base_treino['uf'].unique()

    # 1. Validação das Métricas
    print("\n=== Validação das Métricas (Cross-Validation) ===")
    for uf in ufs:
        train_data = base_treino[base_treino['uf'] == uf].copy()
        X = train_data[features].fillna(0)
        y = np.log(train_data['share'] + 1e-6)
        tscv = TimeSeriesSplit(n_splits=5)
        r2_scores, wmape_scores = [], []
        
        for train_idx, test_idx in tscv.split(X):
            X_cv_train, X_cv_test = X.iloc[train_idx], X.iloc[test_idx]
            y_cv_train, y_cv_test = y.iloc[train_idx], y.iloc[test_idx]
            model = train_optimized_model(X_cv_train, y_cv_train, uf)
            y_pred = np.exp(model.predict(X_cv_test))
            y_true = np.exp(y_cv_test)
            r2_scores.append(r2_score(y_true, y_pred))
            wmape_scores.append(np.sum(np.abs(y_pred - y_true)) / np.sum(y_true))
        
        print(f"{uf}: R² CV = {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
        print(f"{uf}: WMAPE CV = {np.mean(wmape_scores):.4f} ± {np.std(wmape_scores):.4f}")

    # 2. Relação Negativa
    print("\n=== Teste de Relação Negativa ===")
    for uf in ufs:
        df_uf = base_teste[base_teste['uf'] == uf].copy()
        X_base = df_uf[features].fillna(0)
        train_data = base_treino[base_treino['uf'] == uf].copy()
        X_train = train_data[features].fillna(0)
        y_train = np.log(train_data['share'] + 1e-6)
        model = train_optimized_model(X_train, y_train, uf)
        
        df_uf['share_inicial'] = np.exp(model.predict(X_base))
        
        df_pos = df_uf.copy()
        df_pos['preco_medio_in_hl'] *= 1.01
        df_pos['price_squared'] = df_pos['preco_medio_in_hl'] ** 2
        X_pos = df_pos[features].fillna(0)
        df_uf['share_final_pos'] = np.exp(model.predict(X_pos))
        
        df_neg = df_uf.copy()
        df_neg['preco_medio_in_hl'] *= 0.99
        df_neg['price_squared'] = df_neg['preco_medio_in_hl'] ** 2
        X_neg = df_neg[features].fillna(0)
        df_uf['share_final_neg'] = np.exp(model.predict(X_neg))
        
        aumento_preco = (df_uf['share_final_pos'] < df_uf['share_inicial']).mean()
        reducao_preco = (df_uf['share_final_neg'] > df_uf['share_inicial']).mean()
        print(f"{uf}: Aumento de preço reduz share em {aumento_preco:.2%}")
        print(f"{uf}: Redução de preço aumenta share em {reducao_preco:.2%}")
        
        print(f"{uf} - Importância das Features:")
        print(pd.DataFrame({'Feature': features, 'Importance': model.feature_importances_}))

    # 3. Impacto em Produtos sem Variação
    print("\n=== Teste de Impacto em Produtos sem Variação ===")
    for uf in ufs:
        df_uf = base_teste[base_teste['uf'] == uf].copy()
        X_base = df_uf[features].fillna(0)
        model = train_optimized_model(X_train, y_train, uf)
        df_uf['share_inicial'] = np.exp(model.predict(X_base))
        
        df_teste = df_uf.copy()
        first_idx = df_teste.index[0]
        df_teste.loc[first_idx, 'preco_medio_in_hl'] *= 1.01
        df_teste['price_squared'] = df_teste['preco_medio_in_hl'] ** 2
        X_teste = df_teste[features].fillna(0)
        df_uf['share_final'] = np.exp(model.predict(X_teste))
        
        sem_variacao = df_uf[df_uf.index != first_idx]
        perda_volume = (sem_variacao['share_final'] < sem_variacao['share_inicial']).mean()
        print(f"{uf}: Produtos sem variação perdem share em {perda_volume:.2%} (esperado: 0%)")

    # 4. Validação dos Dados
    print("\n=== Validação dos Dados ===")
    for uf in ufs:
        df_uf = base_teste[base_teste['uf'] == uf].copy()
        print(f"{uf} - Estatísticas de preco_medio_in_hl e share:")
        print(df_uf[['preco_medio_in_hl', 'share']].describe())

if __name__ == "__main__":
    validate_model()

Features usadas: ['price_squared', 'price_lag1', 'price_lag2', 'price_index_tt']

=== Validação das Métricas (Cross-Validation) ===
CO: R² CV = -0.0221 ± 0.0321
CO: WMAPE CV = 0.7677 ± 0.0182
MG: R² CV = 0.0388 ± 0.0286
MG: WMAPE CV = 0.7633 ± 0.0160
NE: R² CV = -0.0057 ± 0.0479
NE: WMAPE CV = 0.7477 ± 0.0184
NO: R² CV = -0.0262 ± 0.0261
NO: WMAPE CV = 0.7746 ± 0.0248
RJ_ES: R² CV = 0.0950 ± 0.0606
RJ_ES: WMAPE CV = 0.7465 ± 0.0253
SP: R² CV = -0.0100 ± 0.0274
SP: WMAPE CV = 0.7889 ± 0.0172
SUL: R² CV = -0.0146 ± 0.0316
SUL: WMAPE CV = 0.7648 ± 0.0219

=== Teste de Relação Negativa ===
CO: Aumento de preço reduz share em 15.10%
CO: Redução de preço aumenta share em 15.51%
CO - Importância das Features:
          Feature  Importance
0   price_squared    0.348805
1      price_lag1    0.179420
2      price_lag2    0.172526
3  price_index_tt    0.299248
MG: Aumento de preço reduz share em 15.38%
MG: Redução de preço aumenta share em 14.78%
MG - Importância das Features:
          Feature  

# Elasticidade

In [7]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
np.random.seed(42)

def load_data(path):
    df = pd.read_excel(path, parse_dates=['dt_completa'])
    col_preco = [c for c in df.columns if 'preco' in c.lower() and 'medio' in c.lower() and 'tt' not in c.lower() and 'ts' not in c.lower() and 'tss' not in c.lower()][0]
    if col_preco != 'preco_medio_in_hl':
        df.rename(columns={col_preco: 'preco_medio_in_hl'}, inplace=True)
    
    if 'marca_original' in df.columns and 'marca' not in df.columns:
        df.rename(columns={'marca_original': 'marca', 'sub_marca_original': 'sub_marca'}, inplace=True)
    
    return df

def winsorize_series(series, limits=[0.1, 0.1]):
    return np.clip(series, series.quantile(limits[0]), series.quantile(1 - limits[1]))

def create_features(df, uf=None):
    df = df.copy()
    
    required_cols = ['preco_medio_in_hl', 'distr_numerica_dn', 'share', 'vendas_in_hl', 'price_index_tt']
    for col in required_cols:
        if col not in df.columns:
            raise KeyError(f"Coluna obrigatória ausente: {col}")
    
    df['ln_share'] = np.log(df['share'] + 1e-6)
    df['price_squared'] = df['preco_medio_in_hl'] ** 2
    
    if 'dt_completa' in df.columns and 'uf' in df.columns:
        df = df.sort_values(['uf', 'dt_completa'])
        df['month'] = df['dt_completa'].dt.month
        for lag in [1, 2, 3, 4]:
            df[f'share_lag{lag}'] = df.groupby('uf')['share'].shift(lag)
            df[f'price_lag{lag}'] = df.groupby('uf')['preco_medio_in_hl'].shift(lag)
            df[f'volume_lag{lag}'] = df.groupby('uf')['vendas_in_hl'].shift(lag)
    
    if 'marca' in df.columns:
        df['marca_target_enc'] = df.groupby('marca')['share'].transform('mean')
    if 'segmento' in df.columns:
        df['segmento_target_enc'] = df.groupby('segmento')['share'].transform('mean')
    
    return df

def train_optimized_model(X_train, y_train, uf=None):
    if uf == 'RJ_ES':
        model = CatBoostRegressor(iterations=1000, depth=4, learning_rate=0.05, l2_leaf_reg=5, silent=True)
    elif uf == 'NO':
        model = XGBRegressor(learning_rate=0.1, max_depth=4, n_estimators=500, reg_lambda=15, subsample=0.7, colsample_bytree=0.6, random_state=42)
    else:
        model = XGBRegressor(learning_rate=0.05, max_depth=5, n_estimators=500, reg_lambda=10, random_state=42)
    
    model.fit(X_train, y_train)
    return model

def calculate_elasticities(df, model, uf, features):
    df_uf = df[df['uf'] == uf].copy()
    available_features = [f for f in features if f in df_uf.columns]
    X_base = df_uf[available_features].fillna(0)
    
    df_uf['share_inicial'] = np.exp(model.predict(X_base))
    total_share_inicial = df_uf['share_inicial'].sum()
    df_uf['share_inicial'] = df_uf['share_inicial'] / total_share_inicial
    df_uf['volume_inicial'] = df_uf['vendas_in_hl']
    df_uf['preco_inicial'] = df_uf['preco_medio_in_hl']
    
    resultados = []
    for prod in df_uf['tag_prod'].unique():
        for var in [0.01, -0.01]:
            df_var = df_uf.copy()
            mask_prod = df_var['tag_prod'] == prod
            
            df_var.loc[mask_prod, 'preco_medio_in_hl'] *= (1 + var)
            df_var['preco_final'] = df_var['preco_medio_in_hl']
            df_var['price_squared'] = df_var['preco_final'] ** 2
            df_var['price_index_tt'] = df_var['preco_medio_in_hl'] / df_var['preco_medio_tt']
            
            X_var = df_var[available_features].fillna(0)
            df_var['share_final'] = np.exp(model.predict(X_var))
            
            total_share_final = df_var['share_final'].sum()
            df_var['share_final'] = df_var['share_final'] / total_share_final
            
            total_volume_inicial = df_var['volume_inicial'].sum()
            df_var['volume_final'] = df_var['share_final'] * total_volume_inicial
            df_var['ganho_volume'] = df_var['volume_final'] - df_var['volume_inicial']
            
            mask_no_variation = df_var['tag_prod'] != prod
            total_ganho_variation = df_var.loc[mask_prod, 'ganho_volume'].sum()
            df_var['captura'] = 0  # Inicializa como 0
            
            if total_ganho_variation != 0:
                # Se há ganho/perda, calcula captura normalmente
                df_var.loc[mask_no_variation, 'captura'] = -df_var.loc[mask_no_variation, 'ganho_volume'] / total_ganho_variation
                captura_sum = df_var.loc[mask_no_variation, 'captura'].sum()
                if captura_sum != 0:
                    df_var.loc[mask_no_variation, 'captura'] /= abs(captura_sum)
            else:
                # Se não há ganho/perda, distribui proporcionalmente ao share inicial
                total_share_no_variation = df_var.loc[mask_no_variation, 'share_inicial'].sum()
                if total_share_no_variation > 0:
                    df_var.loc[mask_no_variation, 'captura'] = (
                        df_var.loc[mask_no_variation, 'share_inicial'] / total_share_no_variation * (-1 if var > 0 else 1)
                    )
                # Se não houver produtos não variados, captura permanece 0
            
            df_var['ganho_share'] = df_var['share_final'] - df_var['share_inicial']
            df_var['elasticidade'] = (df_var['ganho_share'] / df_var['share_inicial']) / var
            df_var['impacto_relativo'] = df_var['volume_final'] / df_var['volume_inicial']
            df_var['produto_variado'] = prod
            df_var['variacao'] = var
            
            resultados.append(df_var[[
                'tag_prod', 'uf', 'volume_inicial', 'volume_final', 'share_inicial', 'share_final',
                'preco_inicial', 'preco_final', 'impacto_relativo', 'ganho_volume', 'ganho_share',
                'elasticidade', 'captura', 'produto_variado', 'variacao'
            ]])
    
    return pd.concat(resultados)

def main_elasticities():
    base_elasticidade = load_data("/kaggle/input/data-elasticidade/base_para_elasticidades.xlsx")
    base_elasticidade = create_features(base_elasticidade)
    
    base_features = ['ln_share', 'price_squared', 'price_lag1', 'price_lag2', 'month', 'volume_lag1', 
                     'marca_target_enc', 'segmento_target_enc', 'price_index_tt']
    features = [f for f in base_features if f in base_elasticidade.columns]
    
    elasticidades = []
    ufs = base_elasticidade['uf'].unique()
    
    for uf in ufs:
        print(f"\nCalculando elasticidades para {uf}...")
        
        base_treino = load_data("/kaggle/input/data-elasticidade/base_treino.xlsx")
        base_treino = create_features(base_treino)
        train_data = base_treino[base_treino['uf'] == uf].copy()
        
        for col in ['share', 'preco_medio_in_hl', 'vendas_in_hl']:
            train_data[col] = winsorize_series(train_data[col])
        
        X_train = train_data[features].fillna(0)
        y_train = np.log(train_data['share'] + 1e-6)
        
        model = train_optimized_model(X_train, y_train, uf)
        
        elasticity_data = base_elasticidade[base_elasticidade['uf'] == uf].copy()
        for col in ['share', 'preco_medio_in_hl', 'vendas_in_hl']:
            elasticity_data[col] = winsorize_series(elasticity_data[col])
        
        df_elasticity = calculate_elasticities(elasticity_data, model, uf, features)
        elasticidades.append(df_elasticity)
    
    df_elasticidades = pd.concat(elasticidades)
    df_elasticidades.to_csv("elasticidades_finais.csv", index=False)
    print("\nElasticidades calculadas e salvas em 'elasticidades_finais.csv'.")

if __name__ == "__main__":
    main_elasticities()


Calculando elasticidades para CO...

Calculando elasticidades para MG...

Calculando elasticidades para NE...

Calculando elasticidades para NO...

Calculando elasticidades para RJ_ES...

Calculando elasticidades para SP...

Calculando elasticidades para SUL...

Elasticidades calculadas e salvas em 'elasticidades_finais.csv'.


In [8]:
import pandas as pd

df = pd.read_csv("elasticidades_finais.csv")
for uf in df['uf'].unique():
    print(f"\n=== {uf} ===")
    for var in [0.01, -0.01]:
        for prod in df[df['variacao'] == var]['produto_variado'].unique():
            subset = df[(df['uf'] == uf) & (df['variacao'] == var) & (df['produto_variado'] == prod) & (df['tag_prod'] != prod)]
            captura_total = subset['captura'].sum()
            print(f"Produto variado: {prod}, Variação: {var}, Captura total: {captura_total:.4f}")


=== CO ===
Produto variado: P000000000002014174000000002000156011654, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011655, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011680, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011681, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011682, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011802, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011803, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011804, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011847, Variação: 0.01, Captura total: 1.0000
Produto variado: P000000000002014174000000002000156011848, Variação: 0.01, Captura total: 1.0000
Produto variado: P

In [9]:
df_elasticidade = pd.read_csv('/kaggle/working/elasticidades_finais.csv')
df_elasticidade

,tag_prod,uf,volume_inicial,volume_final,share_inicial,share_final,preco_inicial,preco_final,impacto_relativo,ganho_volume,ganho_share,elasticidade,captura,produto_variado,variacao
0,P000000000002014174000000002000156011654,CO,387.880,400.475586,0.013766,0.013748,872.659070,881.385661,1.032473,12.595586,-1.774170e-05,-0.128883,0.000000,P000000000002014174000000002000156011654,0.01
1,P000000000002014174000000002000156011655,CO,354.880,360.190247,0.012365,0.012365,810.352795,810.352795,1.014963,5.310247,1.545995e-07,0.001250,-0.421555,P000000000002014174000000002000156011654,0.01
2,P000000000002014174000000002000156011680,CO,970.414,936.940735,0.032164,0.032164,1018.921946,1018.921946,0.965506,-33.473265,4.023314e-07,0.001251,2.657282,P000000000002014174000000002000156011654,0.01
3,P000000000002014174000000002000156011681,CO,970.414,937.126404,0.032170,0.032171,899.098743,899.098743,0.965698,-33.287596,4.023314e-07,0.001251,2.642543,P000000000002014174000000002000156011654,0.01
4,P000000000002014174000000002000156011682,CO,21.188,21.925539,0.000753,0.000753,992.103869,992.103869,1.034809,0.737539,9.371433e-09,0.001245,-0.058550,P000000000002014174000000002000156011654,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218145,P000000000002014174000000002000156014493,SUL,227.570,245.166443,0.001952,0.001954,729.922564,729.922564,1.077323,17.596443,1.978711e-06,-0.101390,-0.576506,P000000000002014174000000002000156014622,-0.01
218146,P000000000002014174000000002000156014523,SUL,157.800,169.026733,0.001347,0.001347,794.036755,794.036755,1.071145,11.226733,-5.704351e-09,0.000424,-0.367817,P000000000002014174000000002000156014622,-0.01
218147,P000000000002014174000000002000156014534,SUL,94.770,102.295464,0.000815,0.000815,763.965390,763.965390,1.079408,7.525464,-3.434252e-09,0.000421,-0.246554,P000000000002014174000000002000156014622,-0.01
218148,P000000000002014174000000002000156014554,SUL,96.710,103.795410,0.000827,0.000827,774.149519,774.149519,1.073265,7.085410,-3.434252e-09,0.000415,-0.232137,P000000000002014174000000002000156014622,-0.01
